<a href="https://colab.research.google.com/github/soumabkargit/Extract_Text_Image/blob/master/TesseractMethod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Extraction du montant ttc sur les photos de facture**

#installation de l'ocr Tesseract, prise en charge de Tesseract en Anglais et installation de remotezip. 

[https://github.com/tesseract-ocr/tesseract/wiki](https://)


In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract
!sudo apt-get install tesseract-ocr-eng
!pip install remotezip

##Importation de toutes les dépendances

In [52]:
from matplotlib.image import imread
import numpy as np
from matplotlib import pyplot as plt
import os
import pytesseract
from pytesseract import Output
import matplotlib.patches as patches
from PIL import Image
import re
from pathlib import Path
from shutil import copyfile
from remotezip import RemoteZip
import json
import csv
# Technically not necessary in newest versions of jupyter
%matplotlib inline

## Déclarations et assignations de toutes les variables globales

In [62]:
url = 'https://expressexpense.com/large-receipt-image-dataset-SRD.zip'
lang = 'eng'
image_extract_path = "image_extract"
image_NO_extract_path = "image_NO_extract"
image_NO_read_text_path = "image_NO_read_text"
data = "data"
list_regex = []
regex_1 = r'TOTAL(.)*(\:)*(.)*(\d)'
regex_2 = r'BALANCE(.)*(\:)*(.)*(\d)'
regex_3 = r'AMOUNT(.)*(\:)*(.)*(\d)'
regex_4 = r'GRAND(.)*TOTAL(.)*(\:)*(.)*(\d)'
regex_5 = r'CASH(.)*(\:)*(.)*(\d)'
regex_6 = r'ORDER(.)*BALANCE(.)*(\:)*(.)*(\d)'
regex_amount = r'(\d)'
list_regex.append(regex_1)
list_regex.append(regex_2)
list_regex.append(regex_3)
list_regex.append(regex_4)
list_regex.append(regex_5)

##Implémentation de toutes les méthodes utiles pour l'extraction du montant TTC

In [63]:
def hasNumbers(inputString):
  return any(char.isdigit() for char in inputString)

def add_regex(list_regex, regex):
  list_regex.append(regex)

def extract_amount(regex_amount, string):
  return re.search(regex_amount, string, re.IGNORECASE)

def createFolder(path):
  try:
    os.mkdir(path)
  except OSError:
    print ("la création du dossier %s a échoué" % path)
  else:
    print ("Réussite de la création du dossier %s " % path)

def extracted(url, data):
  try:
    with RemoteZip(url) as zip:
      zip.extractall(path=data)
  except :
    print ("Le zip n'a pas pu être chargé au lien %s " % url)
  
    
def extracted_text_split(imageName, lang):
  im = np.array(Image.open(imageName), dtype=np.uint8)
  extracted_text = pytesseract.image_to_string(im, lang)
  text_split_line = extracted_text.splitlines()
  return text_split_line

def extracted_text_split_file_name(file_path):
  im = file_path
  text_split_text = extracted_text_split(im, lang)
  return text_split_text

def read_all_line_file(text_split_text):
  file_info = {}
  for line in text_split_text:
    for regex in list_regex:
      if regexLine(regex, line):
        if hasNumbers(line):
          file_info['good_info_candidat'] = line
        else:
          file_info['bad_info_candidat'] = line
  return file_info

def regexLine(regex, line):
  return re.search(regex, line, re.IGNORECASE)
    
def copy_file_folder(file_name, path):
  copyfile(file_name, path)

def get_all_names_files(data):
  try :
    list_files = os.listdir(data)
  except OSError:
    print ("La lecture de tout les fichiers dans le dossier %s a échoué" % data)
  else:
    print ("Réussite de la lecture de tout les fichiers dans le dossier %s " % data)
  return list_files

def get_all_amount(data):
  
  files_infos_results = {}
  temp_list_files = {}

  temp_list_files = get_all_names_files(data)

  if len(temp_list_files) == 0 :
    print("Le dossier est vide")
  else :
    for image_filename in temp_list_files:
      file_info_res = get_amount(image_filename)
      files_infos_results[image_filename] = file_info_res
      #arrange_files_quality(file_info_res)
  return files_infos_results

def get_amount(file_name):

   file_info_result = {}

   if len(extracted_text_split_file_name(data + '/' + file_name)) == 0 :
     #print("the file has not been read")
     copy_file_folder(data + '/' + file_name, image_NO_read_text_path + '/' + file_name)
   else :
     text_split_text = extracted_text_split_file_name(data + '/' + file_name)

     if len(read_all_line_file(text_split_text)) == 0 :
       temp_file_info_result = "no information was read"
       #print("no information was read")
     else :
       temp_file_info_result = read_all_line_file(text_split_text)     

     file_info_result[file_name] = file_name
     file_info_result['Amount'] = temp_file_info_result
   arrange_files_quality(file_info_result)
   return file_info_result

def arrange_files_quality(file_info_res):
  try :
    file_info_res[file_name]
    copyfile(data + '/' + file_name, image_extract_path + '/' + file_name)
  except KeyError:
    #print("the amount was not extracted correctly")
    try :
      file_info_res[file_name]
      copyfile(data + '/' + file_name, image_NO_extract_path + '/' + file_name) 
    except KeyError:  
      #print("the file has not been read")
      copyfile(data + '/' + file_name, image_NO_read_text_path + '/' + file_name)
 
    


##Création des dossiers
Dossier de toute les photos de facture

Dossier des factures dont l'extraction du montant ttc a réussi

Dossier des factures dont l'extraction des montant ttc a échoué


In [ ]:
createFolder(image_extract_path)
createFolder(image_NO_extract_path)
createFolder(image_NO_read_text_path)
createFolder(data)

Téléchargement du zip des photos et extraction dans le dossier de toute les factures

In [65]:
extracted(url, data)

Extraction de montant ttc pour une facture

In [77]:
file_name = '1027-receipt.jpg'
res = get_amount(file_name)
print("file name :", res[file_name] ,"\n","Amount :", res['Amount'])

file name : 1027-receipt.jpg 
 Amount : {'good_info_candidat': 'Total $41.31'}


Extraction de montant ttc pour toute les factures

In [71]:
get_all_amount(data)

Successfully to read all files in the folder data 


{'1000-receipt.jpg': {'1000-receipt.jpg': '1000-receipt.jpg',
  'Amount': {'good_info_candidat': 'SUB TOTAL: 51.90'}},
 '1001-receipt.jpg': {'1001-receipt.jpg': '1001-receipt.jpg',
  'Amount': 'no information was read'},
 '1002-receipt.jpg': {'1002-receipt.jpg': '1002-receipt.jpg',
  'Amount': {'good_info_candidat': 'Total 161'}},
 '1003-receipt.jpg': {'1003-receipt.jpg': '1003-receipt.jpg',
  'Amount': 'no information was read'},
 '1004-receipt.jpg': {'1004-receipt.jpg': '1004-receipt.jpg',
  'Amount': {'good_info_candidat': 'TOTAL. $1IS.03'}},
 '1005-receipt.jpg': {'1005-receipt.jpg': '1005-receipt.jpg',
  'Amount': {'good_info_candidat': 'Balance Due 179.94'}},
 '1006-receipt.jpg': {'1006-receipt.jpg': '1006-receipt.jpg',
  'Amount': 'no information was read'},
 '1007-receipt.jpg': {'1007-receipt.jpg': '1007-receipt.jpg',
  'Amount': {'good_info_candidat': 'Total $143.71'}},
 '1008-receipt.jpg': {'1008-receipt.jpg': '1008-receipt.jpg',
  'Amount': {'good_info_candidat': 'TOTAL: $24.

#References

https://nanonets.com/blog/receipt-ocr/?fbclid=IwAR0H4VkHhpimqKIgEUQOUCOJS1mtk6vQREcv3MMiqJ2NpxG9LHZnsHM-aeE


https://www.datacamp.com/community/tutorials/zip-file